In [ ]:
%%bash
jupyter nbconvert clean_patient_info.ipynb --to script

In [ ]:
import sys
sys.path.append('..')

In [ ]:
reload(utils)

In [ ]:
import pandas as pd
import utils
from utils import df_to_dict, filter_by_pt, read_list
from datetime import datetime  
from datetime import timedelta  
import numpy as np

In [ ]:
duration_file = '../../../patient_outcome_info/eeg_data_size_info.csv'

In [ ]:
durations = pd.read_csv(open(duration_file), delimiter='\t')
assert(all(durations['srate']==200))
assert(all(durations['nchans']==19))
durations = durations.drop(labels=['srate', 'nchans'], axis=1)
old_edf_to_nsamples = df_to_dict(durations)

In [ ]:
exclude_patients = read_list('../../../patient_outcome_info/exclude_patient_sids.txt')

In [ ]:
old_to_new_map = pd.read_csv('../../../patient_outcome_info/old_to_new_id.csv')
old_to_new_map.columns = ['old_id', 'new_id']
old_to_new_id = df_to_dict(old_to_new_map)

In [ ]:
old_to_new_edf = pd.read_csv('../../../patient_outcome_info/old_to_new_edf_mapping.csv')
columns = list(old_to_new_edf.columns)
columns[0] = 'pt_edf_id'
old_to_new_edf.columns = columns
old_to_new_edf = old_to_new_edf.drop(labels=['pt_edf_id', 'CSAIL_id', 'new_id', 'error', 'icare_edfs', 'master_edfs',
                                             'exclude'], axis=1)
old_to_new_edf = df_to_dict(old_to_new_edf)

In [ ]:
# invert to create new to old id mapping
new_to_old_id = {}
for old in old_to_new_id:
    new = old_to_new_id[old]
    if new in new_to_old_id:
        new_to_old_id[new].append(old)
    else:
        new_to_old_id[new] = [old]
new_to_old_edf = {}
for old in old_to_new_edf:
    new = old_to_new_edf[old]
    if new in new_to_old_edf:
        new_to_old_edf[new].append(old)
    else:
        new_to_old_edf[new] = [old]

In [ ]:
new_sid: list of timestamps for new sid -> map to old edfs -> duration, is timestamp guessed?, start_index

In [ ]:
merge_info_list = []
patients_with_gaps = []
for new_pt_name in sorted(new_to_old_id.iterkeys(), key=utils.parse_patient_name):
    pt_merge_info_list = []
    new_sid = utils.convert_new_pname_to_new_sid(new_pt_name)
    if new_sid in exclude_patients:
        continue
    old_edfs = filter_by_pt(old_to_new_edf.keys(), new_to_old_id[new_pt_name])
#     new_edfs = filter_by_pt(new_to_old_edf.keys(), new_pt_name)
#     new_for_old = [old_to_new_edf[old] for old in old_edfs]
#     new_for_old = [new for new in new_for_old if not pd.isnull(new)]
#     assert(set(new_for_old)==set(new_edfs))
    for old_edf in old_edfs:
        new_edf = old_to_new_edf[old_edf]
        if pd.isnull(new_edf):
            start_datetime = utils.parse_edf_name(old_edf, ts_datetime=True)[-1]
            timestamp_guessed = True
        else:
            start_datetime = utils.parse_edf_name(new_edf, ts_datetime=True)[-1]
            timestamp_guessed = False
        nsamples = old_edf_to_nsamples[old_edf]
        duration = utils.samples_to_hour(nsamples)
        pt_merge_info_list.append([new_sid, start_datetime, old_edf, duration, nsamples, timestamp_guessed, new_edf])
    pt_merge_info_list.sort()
    for i, pt_merge_info in enumerate(pt_merge_info_list):
        new_sid, start_datetime, old_edf, duration, nsamples, timestamp_guessed, new_edf = pt_merge_info
        end_datetime_obj = start_datetime + timedelta(seconds=utils.samples_to_sec(nsamples))
        end_timestamp = utils.convert_datetime_to_timestamp(end_datetime_obj)
        timestamp = utils.convert_datetime_to_timestamp(start_datetime)
        if i==len(pt_merge_info_list)-1:
            time_to_next = '-'
        else:
            next_start_datetime = pt_merge_info_list[i+1][1]
            time_to_next = (next_start_datetime - end_datetime_obj).total_seconds()/3600.
            if time_to_next > 24 or time_to_next < 0:
                patients_with_gaps.append(new_sid)
        merge_info_list.append([new_sid, timestamp, end_timestamp, old_edf, duration, 
                                nsamples, time_to_next, timestamp_guessed, new_edf])

In [ ]:
merge_info_df = pd.DataFrame(merge_info_list, columns=['sid', 'timestamp', 'end_timestamp', 'csail_edf_name', 
                                                       'duration_(hours)', 'nsamples', 'time_to_next_edf_(hours)',
                                                       'timestamp_guessed', 'new_edf_name'])

In [ ]:
merge_info_df.to_csv('../../../patient_outcome_info/merge_info_w_new_edf_name.csv', index=False)

In [ ]:
merge_info_df

In [ ]:
pd.read_csv('../../../patient_outcome_info/merge_info.csv')

### Analyze gaps

In [ ]:
# analyze gaps

time_to_next = merge_info_df['time_to_next_edf_(hours)']
time_to_next = time_to_next[time_to_next != '-']
with pd.option_context('display.max_rows', None, 'display.max_columns', 3):
    print time_to_next.sort_values()
merge_info_df.loc[merge_info_df['sid'].isin(patients_with_gaps)]

# Everything below is old, unused stuff

In [ ]:
for old_pt_name in old_to_new_id.keys()[-5:]:
    if pd.isnull(old_pt_name):
        print 'null'
    new_pt_name = old_to_new_id[old_pt_name]
    if new_pt_name in exclude_patients:
        continue
    old_edfs = filter_by_pt(old_to_new_edf.keys(), old_pt_name)
    timestamps = []
    for old_edf in old_edfs:
        new_edf = old_to_new_edf[old_edf]
        if pd.isnull(new_edf):
            new_edf = old_edf
        timestamps.append(utils.parse_edf_name(new_edf, ts_datetime=True)[-1])
    edf_nsamples = [durations[old] for old in old_edfs]
    edf_nsecs = [timedelta(seconds=nsamp/200.) for nsamp in edf_nsamples]
    edf_end_time = [nsecs+ts for (ts, nsecs) in zip(timestamps, edf_nsecs)]

In [ ]:
timestamps, edf_end_time
list1, list2 = zip(*sorted(zip(timestamps, edf_end_time)))
start_time_sorted, end_time_sorted = list(list1), list(list2)

In [ ]:
new_to_old_id = {}
for old in old_to_new_id:
    new = old_to_new_id[old]
    if new in new_to_old_id:
        new_to_old_id[new].append(old)
    else:
        new_to_old_id[new] = [old]
new_to_old_edf = {}
for old in old_to_new_edf:
    new = old_to_new_edf[old]
    if new in new_to_old_edf:
        new_to_old_edf[new].append(old)
    else:
        new_to_old_edf[new] = [old]

In [ ]:
for new in new_to_old_id:
    if len(new_to_old_id[new])>1:
        if utils.convert_new_pname_to_new_sid(new) in exclude_patients:
            #print 'exclude', new, new_to_old_id[new]
            continue
        print new, new_to_old_id[new]
# old 210 and 1636 combine to new 210
# old 211 and 1605 are duplicates
# old 213 and 1641 combine to form 213
# old bi 32 and bi 100 combine to form bi 32

In [ ]:
for new in new_to_old_edf:
    if pd.isnull(new):
        continue
    if len(new_to_old_edf[new])>1:
        olds = new_to_old_edf[new]
        if (durations[olds[0]]!=durations[olds[1]]):
            print new, olds, durations[olds[0]], durations[olds[1]]

In [ ]:
old_to_new_id_csv = '../../../patient_outcome_info/old_to_new_id.csv'
exclude_list = '../../../patient_outcome_info/exclude_patient_sids.txt'
class PatientInfoReader:
    def __init__(old_to_new_id_csv='../../../patient_outcome_info/old_to_new_id.csv',
                 exclude_list='../../../patient_outcome_info/exclude_patient_sids.txt',
                 old_to_new_edf_csv='../../../patient_outcome_info/old_to_new_edf_mapping.csv',
                 outcome_csv='../../../patient_outcome_info/clinical_outcomes.csv'
                ):
        # exclude
        self.exclude_patients = read_list(exclude_list)
        
        # old to new ids
        old_to_new_map = pd.read_csv(old_to_new_id_csv)
        old_to_new_map.columns = ['old_id', 'new_id']
        self.old_to_new_id = df_to_dict(old_to_new_map)
        
        # old to new edfs
        old_to_new_edf = pd.read_csv(old_to_new_edf_csv)
        columns = list(old_to_new_edf.columns)
        columns[0] = 'pt_edf_id'
        old_to_new_edf.columns = columns
        old_to_new_edf = old_to_new_edf.drop(columns=['pt_edf_id', 'CSAIL_id', 'new_id', 'error', 'icare_edfs', 'master_edfs',
                                                     'exclude'])
        self.old_to_new_edf = df_to_dict(old_to_new_edf)
        
        # new id to outcomes
        self.outcomes_df = pd.read_csv(outcome_csv, index_col=0)

    def is_excluded(self, patient_filename):
        patient_sid = self.standardize_pt_name(patient_filename)
        return patient_sid in self.exclude_sids
        
    def has_good_outcome(self, patient_filename, vb=False):
        outcome = self.get_outcome(patient_filename, vb)
        if outcome==-1:
            return outcome
        return self.is_good_outcome(outcome)
    
    def is_good_outcome(self, outcome):
        return is_good_outcome(outcome)
    
    def is_bad_outcome(self, outcome):
        return is_bad_outcome(outcome)
    
    def get_patient_info(self, patient_filename):
        patient_sid = self.standardize_pt_name(patient_filename)
        return self.outcomes_df.loc[patient_sid]
    
    def get_outcome(self, patient_filename, vb=False):
        patient_sid = self.standardize_pt_name(patient_filename)
        try:
            outcome = self.outcomes_df['bestCpcBy6Mo'][patient_sid]
        except KeyError:
            if vb:
                if patient_sid in self.exclude_sids:
                    print('{} excluded'.format(patient_filename))
                else:
                    print('KeyError on {}'.format(patient_filename))
            return -1
        if np.isnan(outcome):
            if vb:
                if patient_sid in self.exclude_sids:
                    print('{} excluded'.format(patient_filename))
                else:
                    print('Isnan outcome on {}'.format(patient_filename))
            return -1
        return outcome